In [ ]:
state = {
    'user_query': 'check if my insurance is expired',
    'chat_history': [
        'Marvin: Hi! I am Marvin, a CX Autonomous Agent for your insurance queries. May I have your name, date of birth, and policy number to start with?',
        'User: John Smith 12 Mar 89 987654321',
        'Marvin: Thank you, John. How can I assist you today with your insurance policy?',
        'User: Is my insurance policy still active?',
        'Marvin: Let me check that for you. One moment please.',
        'Marvin: Your insurance policy is active and will expire on 12 Mar 2025. Would you like to renew it now?',
        'User: No, not right now.',
        'Marvin: Understood. If you need any further assistance, feel free to ask.'
    ],
    'policy_number': '987654321',
    'customer_id': 11,
    'policy_id': 12,
    'verification': True,
    'node_data': {},
    'tool': {
        'info': {
            'name': 'policy_status',
            'parameters': {
                'query': 'SELECT ExpiryDate FROM Policies WHERE policy_id=12'
            }
        },
        'agent': 'Policy_status'
    },
    'tool_response': [{'ExpiryDate': '2025-03-12'}],
    'previous_process_state': 'Policy_status',
    'response': 'Understood. If you need any further assistance, feel free to ask.',
    'customer_policy_number': '987654321',
    'comment': 'Verification successful. Customer and Policy details:'
}


In [10]:
verification_prompt='''
You are verification block of chatbot Marvin, your task is to gather information about name, date of birth, and policy number of the user and verify the information using the provided tool.
If some information is missing then you should ask follow-up questions.
Marvin is an insurance bot which helps customers to provide information for their insurance policies and inquiries.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Today's Date: <today date>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "verification",
        "description": "Verifies the user's identity against insurance records.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "The user's full name."
                },
                "date_of_birth": {
                    "type": "string",
                    "description": "The user's date of birth in MM-DD format. Convert the user-provided date to this format."
                },
                "policy_number": {
                    "type": "string",
                    "description": "The user's insurance policy number."
                }
            },
            "required": ["name", "date_of_birth", "policy_number"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "name": "<user's full name>",
                "date_of_birth": "<mm-dd>",
                "policy_number": "<policy number>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1.[{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "name": "<user's full name>",
                "date_of_birth": "<mm-dd>",
                "policy_number": "<policy number>"
            } 
}]
2.[{
    "Action": "Tool call", // Use "Tool call" for calling the verification tool.
    "attribute": {
        "name": "verification",
        "parameters": {
            "name": "<user's full name>",
            "date_of_birth": "<mm-dd>",
            "policy_number": "<policy number>"
        }
    }
    "extracted_entity": {
                "name": "<user's full name>",
                "date_of_birth": "<mm-dd>",
                "policy_number": "<policy number>"
            } 
}]

Note- only respond in JSON output
    - don't explicitly ask user to provide date of birth in certain format 

# '''


In [1]:
policy_status_prompt='''
You are the policy status checker block of chatbot Marvin. Your task is to check the current status of the user's insurance policy using the provided tool.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "policy_status",
        "description": "Checks the current status of the user's insurance policy.",
        "parameters": {
            "type": "object",
            "properties": {
                "policy_number": {
                    "type": "string",
                    "description": "The user's insurance policy number."
                }
            },
            "required": ["policy_number"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "policy_number": "<policy number>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1.[{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]
2.[{
    "Action": "Tool call", // Use "Tool call" for calling the policy status tool.
    "attribute": {
        "name": "policy_status",
        "parameters": {
            "policy_number": "<policy number>"
        }
    }
    "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]

Note- only respond in JSON output

'''


premium_due_date_prompt='''
You are the premium due date checker block of chatbot Marvin. Your task is to check the next premium due date for the user's insurance policy using the provided tool.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "premium_due_date",
        "description": "Provides the next premium due date for the user's insurance policy.",
        "parameters": {
            "type": "object",
            "properties": {
                "policy_number": {
                    "type": "string",
                    "description": "The user's insurance policy number."
                }
            },
            "required": ["policy_number"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "policy_number": "<policy number>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1.[{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]
2.[{
    "Action": "Tool call", // Use "Tool call" for calling the premium due date tool.
    "attribute": {
        "name": "premium_due_date",
        "parameters": {
            "policy_number": "<policy number>"
        }
    }
    "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]

Note- only respond in JSON output

# '''



claim_status_prompt='''
You are the claim status checker block of chatbot Marvin. Your task is to check the status of a claim filed by the user using the provided tool.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "claim_status",
        "description": "Checks the status of a claim filed by the user.",
        "parameters": {
            "type": "object",
            "properties": {
                "claim_number": {
                    "type": "string",
                    "description": "The user's claim number."
                }
            },
            "required": ["claim_number"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "claim_number": "<claim number>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1.[{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "claim_number": "<claim number>"
            } 
}]
2.[{
    "Action": "Tool call", // Use "Tool call" for calling the claim status tool.
    "attribute": {
        "name": "claim_status",
        "parameters": {
            "claim_number": "<claim number>"
        }
    }
    "extracted_entity": {
                "claim_number": "<claim number>"
            } 
}]

Note- only respond in JSON output

# '''



policy_renewal_prompt='''
You are the policy renewal block of chatbot Marvin. Your task is to initiate the process to renew the user's insurance policy using the provided tool.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "policy_renewal",
        "description": "Initiates the process to renew the user's insurance policy.",
        "parameters": {
            "type": "object",
            "properties": {
                "policy_number": {
                    "type": "string",
                    "description": "The user's insurance policy number."
                }
            },
            "required": ["policy_number"]
        }
    }
]
# Output Format:
Return a JSON object with the following structure. If multiple actions are needed, include them in a list, in the order they should be executed:
[
 {
     "Action": <either Response or Tool call as per the output>,
     "attribute": <in case of response return the string and in case of Tool call make it as a JSON>,
     "extracted_entity": {
                "policy_number": "<policy number>"
            } -- fill this placeholder of extracted_entity once you receive any data from user_input else return blank
    }
]

Example of different types of outputs:
1.[{
        "Action": "Response",  // Use "Response" for sending a message to the user.
        "attribute": "<The text of the message to send to the user.>",
        "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]
2.[{
    "Action": "Tool call", // Use "Tool call" for calling the policy renewal tool.
    "attribute": {
        "name": "policy_renewal",
        "parameters": {
            "policy_number": "<policy number>"
        }
    }
    "extracted_entity": {
                "policy_number": "<policy number>"
            } 
}]

Note- only respond in JSON output

# '''



coverage_details_prompt='''
You are the coverage details block of chatbot Marvin. Your task is to provide details about the coverage included in the user's insurance policy using the provided tool.

# Input Format:
    user_input: <The current user's message as a string.>
    chat_history: <A list of previous turns in the conversation.>
    Previous_Extracted_entity: <once all required entity is filled then return the tool call>

# Tool:
[
    {
        "name": "coverage_details",
        "description": "Provides details about the coverage included in the user's insurance policy.",
        "parameters": {
            "type": "object",
            "properties": {
                "policy_number": {
                    "type": "string",
                    "description": "The user's insurance policy number."
                }
            }'''

In [16]:
verification_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''

In [17]:
policy_status_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''


premium_due_date_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''


claim_status_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''

policy_renewal_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''

coverage_details_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''

payment_history_prompt_input='''
<user_input>
{user_input}
</user_input>

<chat_history>
{chat_history}
</chat_history>

<Previous_Extracted_entity>
{extracted_entity}
</Previous_Extracted_entity>

Today's Date:
2025-02-11

# Notes - output the result in Output Format only, DO NOT return anything else.
'''


In [20]:
prompt_stack = {
    "verification": {
        "system": verification_prompt,
        "input": verification_prompt_input
    },
    "policy_status": {
        "system": policy_status_prompt,
        "input": policy_status_prompt_input
    },
    "premium_due_date": {
        "system": premium_due_date_prompt,
        "input": premium_due_date_prompt_input
    },
    "claim_status": {
        "system": claim_status_prompt,
        "input": claim_status_prompt_input
    },
    "policy_renewal": {
        "system": policy_renewal_prompt,
        "input": policy_renewal_prompt_input
    },
    "coverage_details": {
        "system": coverage_details_prompt,
        "input": coverage_details_prompt_input
    }
    
}

In [21]:
# messages = [{'role': 'system',
#                      'content': prompt_stack['verification']['system']},
#                     {'role': 'user',
#                      'content': prompt_stack['verification']['input'].format(user_input=state['user_query'],
#                                                                              chat_history="\n".join(
#                                                                                  state['chat_history'][-5:]),
#                                                                              extracted_entity=str(
#                                                                                  state['node_data']))}]


def create_messages(task, state):
    return [
        {'role': 'system', 'content': prompt_stack[task]['system']},
        {'role': 'user', 'content': prompt_stack[task]['input'].format(
            user_input=state['user_query'],
            chat_history="\n".join(state['chat_history'][-5:]),
            extracted_entity=str(state['node_data'])
        )}
    ]

# Example usage for policy status checker
state = {
    'user_query': 'Is my policy active?',
    'chat_history': ['User: Hi, I need to check my policy status.', 'Marvin: Sure, I can help with that.'],
    'node_data': {}
}
messages = create_messages('policy_status', state)
        

In [22]:
Output = {
    'Action': 'Response',
    'attribute': "I still need the policy number. Please provide it.",
    'extracted_entity': {
        'name': 'John Smith',
        'date_of_birth': '03-12',
        'policy_number': ''
    }
}
 
Output = {
    'Action': 'Response',
    'attribute': "I still need the policy number. Please provide it.",
    'extracted_entity': {
        'name': 'John Smith',
        'date_of_birth': '03-12',
        'policy_number': ''
    }
}

Output = {
    'Action': 'Response',
    'attribute': "I still need your date of birth. Please provide it.",
    'extracted_entity': {
        'name': 'John Smith',
        'date_of_birth': '',
        'policy_number': '987654321'
    }
}

Output = {
    'Action': 'Response',
    'attribute': "I still need your full name. Please provide it.",
    'extracted_entity': {
        'name': '',
        'date_of_birth': '03-12',
        'policy_number': '987654321'
    }
}

Output = {
    'Action': 'Response',
    'attribute': "I still need the claim number. Please provide it.",
    'extracted_entity': {
        'claim_number': ''
    }
}

Output = {
    'Action': 'Response',
    'attribute': "I need your full name, date of birth, and policy number to proceed. Please provide these details.",
    'extracted_entity': {
        'name': '',
        'date_of_birth': '',
        'policy_number': ''
    }
}

Output = {
    'Action': 'Response',
    'attribute': "Thank you for providing the details. Your verification is successful.",
    'extracted_entity': {
        'name': 'John Smith',
        'date_of_birth': '03-12',
        'policy_number': '987654321'
    }
}

Output = {
    'Action': 'Response',
    'attribute': "Your policy is active and will expire on 12 Mar 2025.",
    'extracted_entity': {
        'policy_number': '987654321'
    }
}

Output = {
    'Action': 'Response',
    'attribute': "Your policy has expired. Would you like to renew it now?",
    'extracted_entity': {
        'policy_number': '987654321'
    }
}


In [24]:
import json
data_final = {'data':[]}
data = {
    'prompt': [messages],
    'output': [Output]
}

In [25]:
data_final['data'].append(data)

In [26]:
json_output = json.dumps(data, indent=4)
print(json_output)

{
    "prompt": [
        [
            {
                "role": "system",
                "content": "\nYou are the policy status checker block of chatbot Marvin. Your task is to check the current status of the user's insurance policy using the provided tool.\n\n# Input Format:\n    user_input: <The current user's message as a string.>\n    chat_history: <A list of previous turns in the conversation.>\n    Previous_Extracted_entity: <once all required entity is filled then return the tool call>\n\n# Tool:\n[\n    {\n        \"name\": \"policy_status\",\n        \"description\": \"Checks the current status of the user's insurance policy.\",\n        \"parameters\": {\n            \"type\": \"object\",\n            \"properties\": {\n                \"policy_number\": {\n                    \"type\": \"string\",\n                    \"description\": \"The user's insurance policy number.\"\n                }\n            },\n            \"required\": [\"policy_number\"]\n        }\n    

In [2]:
process_identification_prompt = '''
Your role is to identify the user's intent and guide the conversation towards one of the supported travel-related processes. You are the *first* step in the conversation flow.

**Supported Processes:**

1. Booking: The user is making a new booking (e.g., "make a booking," "book an appointment," "schedule my meeting").
2. Rebooking: The user is rescheduling an existing booking (e.g., "rebook my flight," "change my appointment time").
3. Cancellation: The user wants to cancel a service, booking, or other request (e.g., "cancel my booking," "I want to cancel the service").
4. Modification: The user wants to make changes to an existing booking or service (e.g., "change my reservation," "update my contact information").
5. Extra Services: The user is requesting, upgrading, or adding additional services or features to an existing booking (e.g., "add a meal," "upgrade my seat," "request early check-in").

**Handling "Other" Intents:**

If the user's intent does *not* clearly fall into one of the supported categories, classify it as **`Other`**. This includes:

* Greetings: (e.g., "Hi," "Hello," "Good morning")
* Thanks/Gratitude: (e.g., "Thank you," "Thanks," "Appreciate it")
* General Inquiries/Small Talk: Questions or statements not directly related to the supported processes.
* Unclear Requests: If the user's input is ambiguous or nonsensical.

For the **`Other`** intent:

* Generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported travel-related processes.

**Input:**

* `user_input`: The user's most recent message (string).
* `chat_history`: The full conversation history (list of strings, alternating between user and assistant messages). Use the `chat_history` to help determine the intent, especially if the `user_input` is short or ambiguous.

# Output Format 
The response should be a LIST of JSON objects with the following structure:
'''

In [7]:
pip install botbuilder-core


Note: you may need to restart the kernel to use updated packages.


In [6]:
# !pip install  botbuilder
from botbuilder.core import TurnContext, ActivityHandler

class MyBot(ActivityHandler):
    async def on_message_activity(self, turn_context: TurnContext):
        user_input = turn_context.activity.text
        chat_history = [
            "assistant: Hello, my name is Marvis. To assist you, I will need your policy number",
            "user: Hi.",
            "user: Policy number: 12345.",
            "assistant: How can I help you?",
            "user: I am trying to check the status of my insurance claim",
            "user: Can you please help with that.",
            "assistant: Your claim is currently being processed. Please allow a few more days for completion."
        ]

        intent = self.identify_intent(user_input, chat_history)
        if intent in ["Policy Status", "Policy Adjustment", "Policy Cancellation", "Policy Modification", "Additional Coverage Options"]:
            tool_call = {
                "Action": "Tool call",
                "attribute": {
                    "name": intent.lower(),
                    "arguments": {
                        "user_input": user_input,
                        "chat_history": chat_history
                    }
                }
            }
            await turn_context.send_activity(f"Tool call: {tool_call}")
        else:
            response = {
                "Action": "Response",
                "attribute": "I'm here to help with Policy Enrollment ,Policy Adjustment, Policy Cancellation, Policy Modification,Additional Coverage Options. How can I assist you today?"
            }
            await turn_context.send_activity(response["attribute"])

    def identify_intent(self, user_input, chat_history):
        if "Policy Status" in user_input.lower():
            return "Status"
        elif "Policy Adjustment" in user_input.lower():
            return "Adjustment"
        elif "Policy Cancellation" in user_input.lower():
            return "Cancellation"
        elif "Policy Modification" in user_input.lower():
            return "Modification"
        elif "add" in user_input.lower() or "upgrade" in user_input.lower():
            return "Additional Coverage Options"
        else:
            return "Other"


In [8]:
[
    {
        "Action": "Tool call",
        "attribute": {
            "name": "Status",
            "arguments": {
                "user_input": "The clain should be given I think",
                "chat_history": [
                    "assistant: Hello, my name is Marvis. To assist you, I will need your policy number",
                    "user: Hi.",
                    "user: Policy number: 12345.",
                    "assistant: How can I help you?",
                    "user: I am trying to check the status of my insurance claim",
                    "user: Can you please help with that.",
                    "assistant: Your claim is currently being processed. Please allow a few more days for completion."
                ]
            }
        }
    }
]


[{'Action': 'Tool call',
  'attribute': {'name': 'Status',
   'arguments': {'user_input': 'The clain should be given I think',
    'chat_history': ['assistant: Hello, my name is Marvis. To assist you, I will need your policy number',
     'user: Hi.',
     'user: Policy number: 12345.',
     'assistant: How can I help you?',
     'user: I am trying to check the status of my insurance claim',
     'user: Can you please help with that.',
     'assistant: Your claim is currently being processed. Please allow a few more days for completion.']}}}]

In [33]:
insurance_tool_call = {
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_insurance_status\", \"description\": \"Check the status of an insurance claim\", \"parameters\": {\"type\": \"object\", \"properties\": {\"policy_number\": {\"type\": \"string\", \"description\": \"The policy number to check the status for\"}}, \"required\": [\"policy_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Hi, I need to check the status of my insurance claim. </user_input>"
        },
        {
            "role": "assistant",
            "content": {
                "Action": "Response",
                "attribute": "Sure, I can help with that. Could you please provide me with your policy number?"
            }
        },
        {
            "role": "user",
            "content": "<user_input> Yes, it's 12345. </user_input>"
        }
    ],
    "output": [
        {
            "Action": "Tool call",
            "attribute": {
                "name": "check_insurance_status",
                "arguments": {
                    "policy_number": "12345"
                }
            }
        }
    ]
}


In [34]:
insurance_tool_call

{'input': [{'role': 'system',
   'content': 'You are a helpful assistant with access to functions and tools. Use them if required.\n[{"name": "check_insurance_status", "description": "Check the status of an insurance claim", "parameters": {"type": "object", "properties": {"policy_number": {"type": "string", "description": "The policy number to check the status for"}}, "required": ["policy_number"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     "Action":<either Response or Tool call as per the output>\n     "attribute":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n'},
  {'role': 'user',
   'content': '<user_input> Hi, I need to check the status of my insurance claim. </user_input>'},
  {'role': 'assistant',
   'content': {'Action': 'Response',
    'attribute': 'S

In [35]:
insurance_response_generate = {
    "input": [
        {
            "role": "system",
            "content": "You are a helpful assistant with access to functions and tools. Use them if required.\n[{\"name\": \"check_insurance_status\", \"description\": \"Check the status of an insurance claim\", \"parameters\": {\"type\": \"object\", \"properties\": {\"policy_number\": {\"type\": \"string\", \"description\": \"The policy number to check the status for\"}}, \"required\": [\"policy_number\"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     \"Action\":<either Response or Tool call as per the output>\n     \"attribute\":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n"
        },
        {
            "role": "user",
            "content": "<user_input> Can you help me book a flight? </user_input>"
        }
    ],
    "output": [
        {
            "Action": "Response",
            "attribute": "I'm sorry, but as an AI, I don't have the capability to perform external tasks such as booking flights. My current function allows me to check the status of your insurance claim. If you need assistance with that, please provide your policy number."
        }
    ]
}


In [36]:
insurance_response_generate

{'input': [{'role': 'system',
   'content': 'You are a helpful assistant with access to functions and tools. Use them if required.\n[{"name": "check_insurance_status", "description": "Check the status of an insurance claim", "parameters": {"type": "object", "properties": {"policy_number": {"type": "string", "description": "The policy number to check the status for"}}, "required": ["policy_number"]}}]\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n    {\n     "Action":<either Response or Tool call as per the output>\n     "attribute":< in case of response return the string and in case of Tool call make it as a JSON>\n    }\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order.\n \n'},
  {'role': 'user',
   'content': '<user_input> Can you help me book a flight? </user_input>'}],
 'output': [{'Action': 'Response',
   'attribute': "I'm sorry, but as an AI, I don't have the capabili

In [37]:
insurance_process_identification = {
    "input": [
        {
            "role": "system",
            "content": "Your role is to identify the user's intent and guide the conversation towards one of the supported insurance-related processes. You are the *first* step in the conversation flow.\n\n**Supported Processes:**\n\n1.  Policy Enrollment: The user is enrolling in a new insurance policy (e.g., \"enroll in a policy,\" \"sign up for insurance\").\n2.  Policy Adjustment: The user is adjusting an existing policy (e.g., \"adjust my coverage,\" \"change my policy details\").\n3.  Policy Cancellation: The user wants to cancel an insurance policy (e.g., \"cancel my policy,\" \"I want to terminate my insurance\").\n4.  Policy Modification: The user wants to make changes to an existing policy (e.g., \"update my policy information,\" \"modify my coverage\").\n5.  Additional Coverage Options: The user is requesting, upgrading, or adding additional coverage to an existing policy (e.g., \"add dental coverage,\" \"upgrade my health insurance\").\n\n**Handling \"Other\" Intents:**\n\nIf the user's intent does *not* clearly fall into one of the supported categories, classify it as **`Other`**. This includes:\n\n*   **Greetings:** (e.g., \"Hi,\" \"Hello,\" \"Good morning\")\n*   **Thanks/Gratitude:** (e.g., \"Thank you,\" \"Thanks,\" \"Appreciate it\")\n*   **General Inquiries/Small Talk:** Questions or statements not directly related to the supported processes.\n*   **Unclear Requests:**  If the user's input is ambiguous or nonsensical.\n\nFor the **`Other`** intent:\n\n*   generate a polite, helpful response that acknowledges the user's input and gently steers the conversation back towards the supported insurance-related processes.\n\n**Input:**\n\n*   **`user_input`:** The user's most recent message (string).\n*   **`chat_history`:** The full conversation history (list of strings, alternating between user and assistant messages).  Use the `chat_history` to help determine the intent, especially if the `user_input` is short or ambiguous.\n\n# Output Format \nThe response should be a LIST of JSON object with the following structure:\n```\n[\n{\n    \"Action\":\"<Intent>\",\n    \"attribute\":\"<Response or null>\"\n}\n]\n```\nNote- if the output has multiple output then stack up the JSON object in the LIST in FIFO order."
        },
        {
            "role": "user",
            "content": "<user_input>\nuser: I believe my policy covers this. \n</user_input>\n\n<chat_history>\nassistant: Hello, my name is Jeni. To assist you, I will need your policy number. \nuser: Hi. \nuser: Policy number: 89sjb5. \nassistant: How can I help you today? \nuser: I am trying to check the status of my insurance claim. \nuser: Can you please help with that? \nassistant: Your claim is currently being processed. Please allow a few more days for completion. \n</chat_history>"
        }
    ],
    "output": [
        {
            "Action": "Policy Adjustment",
            "attribute": None
        }
    ]
}


In [38]:
insurance_process_identification

{'input': [{'role': 'system',
   'content': 'Your role is to identify the user\'s intent and guide the conversation towards one of the supported insurance-related processes. You are the *first* step in the conversation flow.\n\n**Supported Processes:**\n\n1.  Policy Enrollment: The user is enrolling in a new insurance policy (e.g., "enroll in a policy," "sign up for insurance").\n2.  Policy Adjustment: The user is adjusting an existing policy (e.g., "adjust my coverage," "change my policy details").\n3.  Policy Cancellation: The user wants to cancel an insurance policy (e.g., "cancel my policy," "I want to terminate my insurance").\n4.  Policy Modification: The user wants to make changes to an existing policy (e.g., "update my policy information," "modify my coverage").\n5.  Additional Coverage Options: The user is requesting, upgrading, or adding additional coverage to an existing policy (e.g., "add dental coverage," "upgrade my health insurance").\n\n**Handling "Other" Intents:**\n\n